In [2]:
import httpx
import json

# --- Configuration ---
config = json.load(open("config.json"))
api_key = open('botify_token.txt').read().strip()
org_slug = config['org']
project_slug = config['project']
headers = {"Authorization": f"Token {api_key}", "Content-Type": "application/json"}

# --- List Collections ---
def list_collections(org, project, key):
    collections_url = f"https://api.botify.com/v1/projects/{org}/{project}/collections"
    print(f"Fetching collections for {org}/{project}...")
    try:
        response = httpx.get(collections_url, headers=headers, timeout=60.0)
        response.raise_for_status()
        collections_data = response.json()
        print("Available Collections:")
        for collection in collections_data:
            print(f"- ID: {collection.get('id')}, Name: {collection.get('name')}")
        return collections_data
    except Exception as e:
        print(f"Error fetching collections: {e}")
        return None

print("\n--- Step 1: Finding the Log Collection Name ---")
available_collections = list_collections(org_slug, project_slug, api_key)
# **ACTION**: Examine the output above. Identify the collection ID that corresponds to web logs.
# It might be named something like 'logs', 'weblogs', 'log_analytics', or specific to the source (e.g., 'cdn_logs').
# Let's assume you identify it as 'ACTUAL_LOG_COLLECTION_NAME' for the next step.


--- Step 1: Finding the Log Collection Name ---
Fetching collections for ariat-international-org/www.ariat.com...
Available Collections:
- ID: pageworkers_tags_logs, Name: Pageworkers Tags Logs
- ID: global, Name: URL Scheme and Segmentation
- ID: logs, Name: Logs
- ID: actionboard_ml.20250409, Name: ActionBoard ML
- ID: generic.ActionBoardML.20250409, Name: ActionBoard ML
- ID: generic.QueryMaskML.20250409, Name: Page Category
- ID: crawl.20250409, Name: 2025 Apr. 9th
- ID: search_engines_orphans.20250409, Name: Search Engines Orphans
- ID: actionboard_ml.20250326, Name: ActionBoard ML
- ID: generic.ActionBoardML.20250326, Name: ActionBoard ML
- ID: generic.QueryMaskML.20250326, Name: Page Category
- ID: actionboard_ml.20250312, Name: ActionBoard ML
- ID: actionboard_ml.20250212, Name: ActionBoard ML
- ID: actionboard_ml.20250226, Name: ActionBoard ML
- ID: generic.ActionBoardML.20250312, Name: ActionBoard ML
- ID: generic.QueryMaskML.20250312, Name: Page Category
- ID: generic.Actio

In [4]:
import httpx
import json
import os
# No typing imports needed

# --- Configuration ---
CONFIG_FILE = "config.json"
TOKEN_FILE = "botify_token.txt"
# The specific collection ID we are looking for
TARGET_LOG_COLLECTION_ID = "logs"

# --- !!! EASY OVERRIDE SECTION !!! ---
# Set these variables to directly specify org/project, bypassing config.json
# Leave as None to use config.json or prompts.
# https://app.botify.com/michaellevin-org/mikelev.in
ORG_OVERRIDE = "michaellevin-org"
PROJECT_OVERRIDE = "mikelev.in"
# Example:
# ORG_OVERRIDE = "my-direct-org"
# PROJECT_OVERRIDE = "my-direct-project.com"
# --- END OVERRIDE SECTION ---


# --- Helper Functions ---

def load_api_key():
    """Loads the API key from the token file. Returns None on error."""
    try:
        if not os.path.exists(TOKEN_FILE):
            print(f"Error: Token file '{TOKEN_FILE}' not found.")
            return None
        with open(TOKEN_FILE) as f:
            api_key = f.read().strip()
            if not api_key:
                print(f"Error: Token file '{TOKEN_FILE}' is empty.")
                return None
        return api_key
    except Exception as e:
        print(f"Error loading API key: {e}")
        return None

def load_org_project_from_config():
    """Loads org and project from config file. Returns (None, None) on error or if missing."""
    org_config = None
    project_config = None
    try:
        if os.path.exists(CONFIG_FILE):
            with open(CONFIG_FILE) as f:
                config_data = json.load(f)
                org_config = config_data.get('org')
                project_config = config_data.get('project')
        # No error if file doesn't exist, just return None
    except json.JSONDecodeError:
        print(f"Warning: '{CONFIG_FILE}' contains invalid JSON.")
    except Exception as e:
        print(f"Warning: Could not load org/project from {CONFIG_FILE}: {e}")
    return org_config, project_config

def get_api_headers(api_key):
    """Returns standard API headers."""
    return {
        "Authorization": f"Token {api_key}",
        "Content-Type": "application/json"
    }

def check_if_log_collection_exists(org_slug, project_slug, api_key):
    """
    Checks if a collection with ID 'logs' exists for the given org and project.
    Returns True if found, False otherwise or on error.
    """
    if not org_slug or not project_slug or not api_key:
        print("Error: Org slug, project slug, and API key are required for check.")
        return False

    collections_url = f"https://api.botify.com/v1/projects/{org_slug}/{project_slug}/collections"
    headers = get_api_headers(api_key)

    print(f"\nChecking for collection '{TARGET_LOG_COLLECTION_ID}' in {org_slug}/{project_slug}...")

    try:
        response = httpx.get(collections_url, headers=headers, timeout=60.0)

        if response.status_code == 401:
            print("Error: Authentication failed (401). Check your API token.")
            return False
        elif response.status_code == 403:
             print("Error: Forbidden (403). You may not have access to this project or endpoint.")
             return False
        elif response.status_code == 404:
             print("Error: Project not found (404). Check org/project slugs.")
             return False

        response.raise_for_status() # Raise errors for other bad statuses (like 5xx)
        collections_data = response.json()

        if not isinstance(collections_data, list):
             print(f"Error: Unexpected API response format. Expected a list.")
             return False

        for collection in collections_data:
            if isinstance(collection, dict) and collection.get('id') == TARGET_LOG_COLLECTION_ID:
                print(f"Success: Found collection with ID '{TARGET_LOG_COLLECTION_ID}'.")
                return True

        print(f"Result: Collection with ID '{TARGET_LOG_COLLECTION_ID}' was not found in the list.")
        return False

    except httpx.HTTPStatusError as e:
         print(f"API Error checking collections: {e.response.status_code}")
         return False
    except httpx.RequestError as e:
        print(f"Network error checking collections: {e}")
        return False
    except json.JSONDecodeError:
        print("Error: Could not decode the API response as JSON.")
        return False
    except Exception as e:
        print(f"An unexpected error occurred during check: {e}")
        return False

# --- Main Function ---

def run_check(org_override=None, project_override=None):
    """
    Orchestrates the check for the 'logs' collection.
    Uses override values if provided, otherwise falls back to config file, then prompts.
    """
    print("Starting Botify Log Collection Check...")

    # 1. Load API Key (Essential)
    api_key = load_api_key()
    if not api_key:
        print("Cannot proceed without a valid API key.")
        return # Exit the function

    # 2. Determine Org and Project to use
    org_config, project_config = load_org_project_from_config()

    # Apply overrides if they exist
    org_to_use = org_override if org_override is not None else org_config
    project_to_use = project_override if project_override is not None else project_config

    # If still missing after config and overrides, prompt the user
    if not org_to_use:
        print(f"Organization slug not found in config or override.")
        org_to_use = input("Enter the organization slug: ").strip()

    if not project_to_use:
        print(f"Project slug not found in config or override.")
        project_to_use = input("Enter the project slug: ").strip()

    # Final check before running API call
    if not org_to_use or not project_to_use:
        print("Organization and Project slugs are required to run the check. Exiting.")
        return

    # 3. Run the core check function
    has_logs = check_if_log_collection_exists(org_to_use, project_to_use, api_key)

    # 4. Report Final Result
    print("\n--- Check Complete ---")
    if has_logs:
        print(f"The project '{org_to_use}/{project_to_use}' appears to HAVE a '{TARGET_LOG_COLLECTION_ID}' collection available.")
    else:
        print(f"The project '{org_to_use}/{project_to_use}' does NOT appear to have a '{TARGET_LOG_COLLECTION_ID}' collection available (or an error occurred).")


# --- Script Execution ---
if __name__ == "__main__":
    # Call the main function, passing the override values defined at the top
    run_check(org_override=ORG_OVERRIDE, project_override=PROJECT_OVERRIDE)

Starting Botify Log Collection Check...

Checking for collection 'logs' in michaellevin-org/mikelev.in...
Result: Collection with ID 'logs' was not found in the list.

--- Check Complete ---
The project 'michaellevin-org/mikelev.in' does NOT appear to have a 'logs' collection available (or an error occurred).
